# HDDA. Lab6. Clustering

## Hierarchical

Complete linkage is one implementation of hierarchical agglomerative clustering. The principle of hierarchical agglomerative clustering is to start with a singleton cluster, and clusters are iteratively merged until one single cluster remains. This results in a "cluster tree," which is also called dendrogram. The opposite approach -- starting with one cluster and divide into clusters until only singleton clusters remain - is called divisive hierarchical clustering.

**Algo:**

- Compute a distance or similarity matrix.
- Each data point is represented as a singleton cluster.
- Repeat
    - Merge two closest clusters (e.g., based on distance between most similar or dissimilar members).
    - Update the distance (or similarity) matrix.
- Until one single cluster remains.

*The single linkage* algorithm compares the two most similar members instead of the most dissimilar ones

*Complete linkage* compares the most dissimilar members between clusters in each iteration. The two clusters which have the most similar dissimilar members are merged into a new cluster.

### Data

In [ ]:
import pandas as pd
import numpy as np

np.random.seed(123)

variables = ['X', 'Y', 'Z']
labels = ['ID_0','ID_1','ID_2','ID_3','ID_4']

X = np.random.random_sample([5,3])*10
df = pd.DataFrame(X, columns=variables, index=labels)
df

First, we calculate the pair-wise distances for every row (i.e, between every sample across the different variables). We will use the default euclidean distance measure. 
 In addition, we use the squareform function to return a symmetrical matrix of the pair-wise distances.

In [ ]:
from scipy.spatial.distance import pdist,squareform

row_dist = pd.DataFrame(squareform(pdist(df, metric='euclidean')), columns=labels, index=labels)
row_dist

#### Squareform distance matrix


When we apply the complete linkage agglomeration to our clusters, the linkage function returns a so-called linkage matrix. This linkage matrix consists of several rows where each row consists of 1 merge. The first and second column denote the most dissimilar members in each cluster, and the third row reports the distance between those members. The last column returns the count of members in the clusters.

In [ ]:
from scipy.cluster.hierarchy import linkage

row_clusters = linkage(row_dist, method='complete', metric='euclidean')
pd.DataFrame(row_clusters, 
             columns=['row label 1', 'row label 2', 'distance', 'no. of items in clust.'],
             index=['cluster %d' %(i+1) for i in range(row_clusters.shape[0])])

#### Condensed distance matrix 

Thus, we can either pass a condensed distance matrix (upper triangular) from the pdist function, or we can pass the "original" data array and define the 'euclidean' metric as function argument in linkage. However, we shouldn't pass the squareform distance metrics, which would yield incorrect distance values although the overall clustering could be the same.

In [ ]:
row_clusters = linkage(pdist(df, metric='euclidean'), method='complete')
pd.DataFrame(row_clusters, 
             columns=['row label 1', 'row label 2', 'distance', 'no. of items in clust.'],
             index=['cluster %d' %(i+1) for i in range(row_clusters.shape[0])])

#### Input sample matrix


In [ ]:
row_clusters = linkage(df.values, method='complete', metric='euclidean')
pd.DataFrame(row_clusters, 
             columns=['row label 1', 'row label 2', 'distance', 'no. of items in clust.'],
             index=['cluster %d' %(i+1) for i in range(row_clusters.shape[0])])

In [ ]:
from scipy.cluster.hierarchy import dendrogram

row_dendr = dendrogram(row_clusters, labels=labels)

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(111)

cax = ax.matshow(df, interpolation='nearest', cmap='hot_r')
fig.colorbar(cax)

ax.set_xticklabels([''] + list(df.columns))
ax.set_yticklabels([''] + list(df.index))

plt.show()

In [ ]:
from scipy.cluster import hierarchy
# makes dendrogram black (1)
hierarchy.set_link_color_palette(['black'])

# plot row dendrogram
fig = plt.figure(figsize=(8,8))
axd = fig.add_axes([0.09,0.1,0.2,0.6])
row_dendr = dendrogram(row_clusters, orientation='left', 
                   color_threshold=np.inf, ) # makes dendrogram black (2))

# reorder data with respect to clustering
df_rowclust = df.ix[row_dendr['leaves'][::-1]]

axd.set_xticks([])
axd.set_yticks([])


# remove axes spines from dendrogram
for i in axd.spines.values():
        i.set_visible(False)

# reorder rows with respect to the clustering
df_rowclust = df.ix[row_dendr['leaves'][::-1]]
        
# plot heatmap
axm = fig.add_axes([0.26,0.1,0.6,0.6]) # x-pos, y-pos, width, height
cax = axm.matshow(df_rowclust, interpolation='nearest', cmap='hot_r')
fig.colorbar(cax)
axm.set_xticklabels([''] + list(df_rowclust.columns))
axm.set_yticklabels([''] + list(df_rowclust.index))

plt.show()

## K-means


In [ ]:
%matplotlib inline
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from numpy.random import RandomState
from sklearn import datasets

In [ ]:
from sklearn.datasets.samples_generator import make_blobs
np.random.seed(0)
centers = [[1, 1], [-1, -1]]
n_clusters = len(centers)
X, labels_true = make_blobs(n_samples=3000, 
                            centers=centers, 
                            cluster_std=0.5)

In [ ]:
def plot_cluster_data(X, c=[1]*X.shape[0], mu=None):
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(1, 1, 1)
    if len(np.unique(c)) == 1:
        ax.plot(X[:,0], X[:,1], 'o')
    else:
        ix = np.where(c==1)
        ax.plot(X[ix,0], X[ix,1], 'o', 
                markerfacecolor='red')
        ax.plot(mu[0,0], mu[0,1], 'o', 
                markerfacecolor='red', 
                markersize=12)
        ix = np.where(c==0)
        ax.plot(X[ix,0], X[ix,1], 'o', 
                markerfacecolor='green')
        ax.plot(mu[1,0], mu[1,1], 'o', 
                markerfacecolor='green', 
                markersize=12)
    if not mu is None:
        ax.plot(mu[0,0], mu[0,1], 'o', 
                markerfacecolor='red', 
                markersize=12)
        ax.plot(mu[1,0], mu[1,1], 'o', 
                markerfacecolor='green', 
                markersize=12)        
    plt.show()

In [ ]:
plot_cluster_data(X)

In [ ]:
clst = KMeans(n_clusters=2, random_state=2342)
clst.fit(X)
mu = clst.cluster_centers_
plot_cluster_data(X, mu = mu)

In [ ]:
def update_labels(X, mu):
    c = np.argmax(np.c_[np.sum(np.power(X - mu[0,:], 2), axis=1), 
                        np.sum(np.power(X - mu[1,:], 2), axis=1)], 
                  axis=1)
    return c

def update_cluster_centers(X, c):
    ix = np.where(c==1)
    mu[0,:] = np.mean(X[ix,:], axis=1)
    ix = np.where(c==0)
    mu[1,:] = np.mean(X[ix,:], axis=1)
    return mu

In [ ]:
k = 2
mu = np.array([[2.0,-1.0], [-2.0,1.0]])
plot_cluster_data(X, mu=mu)

In [ ]:
niter = 2
for it in range(niter):
    print('Iteration ' + str(it) + ':')
    c = update_labels(X, mu)
    print('...updating labels:')
    plot_cluster_data(X, c=c, mu=mu)
    print('...updating centers:')    
    mu = update_cluster_centers(X, c)
    plot_cluster_data(X, mu=mu)

## DBSCAN

In [ ]:
%matplotlib inline
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from numpy.random import RandomState
from sklearn import datasets

In [ ]:
points, labels = datasets.make_moons(n_samples=100, noise=.05)
palette = np.array(sns.color_palette("hls", 8))
plt.scatter(*points.T, color=palette[labels])

In [ ]:
moons = KMeans(n_clusters=2, random_state=RandomState(42))
moons.fit(points)
plt.scatter(*points.T, color=palette[moons.labels_])

DBSCAN (Density-Based Spatial Clustering of Applications with Noise) is a density-based clustering algorithm, introduced [Ester et al. (1996)](https://www.aaai.org/Papers/KDD/1996/KDD96-037.pdf).

DBSCAN infers the number of clusters from the dataset, allowing it to discover clusters of arbitrary shape. It establishes a **neighborhood size**, and assigns points to categories based on their relationship to other points, conditioned on this neighborhood size. 

This confers several advantages:

- Allows for more complex cluster shapes
- K does not need to be specified
- Automatically finds outliers

While we don't need to choose K, we do need to select a distance function for quantifying **dissimilarity** between points.



2. Connect core points into clusters

3. Assign boundary points to clusters

The steps to the DBSCAN algorithm are:

1. **Determine the type of a new point**

    - core
    - boundary
    - outlier

  We randomly pick that has not yet been assigned to a cluster, or been designated as an outlier. For this point, we determine its neighborhood. If it is a **core point**, we seed a cluster around it, otherwise label the point as an **outlier**.

2. **Expand the new cluster by adding all reachable points**. Once we find a core point (and hence, a cluster), find all points that are reachable based in the neighborhood and add them to the cluster. If a point previously found to be an outlier is included, change its status to a **border point**.

3. Repeat these steps until all points are either assigned to a cluster or designated as an outlier.

The `DBSCAN` class in `scikit-learn` is a straightforward implementation of this algorithm, and requires three main arguments:

- `eps` defines the size of the neighborhood around each point

- `min_samples` is the number of points that needs to be within the neigborhood for a point to be considered a core point; density level threshold

- `metric` is either a callable function or a string corresponding to a built-in distance metric


In [ ]:
from sklearn.cluster import DBSCAN

dbscan_moons = DBSCAN(eps=0.4, min_samples=11)
dbscan_moons.fit(points)
dbscan_moons.labels_

In [ ]:
plt.scatter(*points.T, color=palette[dbscan_moons.labels_])

## Gaussian Mixture Model

In [ ]:
np.random.seed(0)
n_samples = 1000
X1 = 2.0*np.random.randn(n_samples, 2) + np.array([5, 3])
C = np.array([[0., -0.5], [3.5, .7]])
X2 = np.dot(np.random.randn(n_samples, 2), C)
X_train = np.vstack([X1, X2])

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1)
ax.plot(X_train[:,0], X_train[:,1], 'o')
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
from sklearn.mixture import GMM

In [ ]:
np.random.seed(1)
model = GMM(n_components=2, covariance_type='full')
model.fit(X_train)

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1)
ax.plot(X_train[:,0], X_train[:,1], 'o', alpha=.1, ms=5)

for i in range(2):    
    mu = model.means_[i]
    sigma = model.covars_[i]
    sigma_inv = np.linalg.inv(sigma)
    sigma_det = np.linalg.det(sigma)
    x = np.linspace(-15.0, 15.0)
    y = np.linspace(-4.0, 10.0)
    X, Y = np.meshgrid(x, y)
    XX = np.array([X.ravel(), Y.ravel()]).T
    XX = np.dot(np.dot(XX - mu, sigma_inv), 
                np.transpose(XX - mu))
    P = np.exp(-0.5*np.diagonal(XX))/(2*np.pi*sigma_det**0.5)
    P = P.reshape(X.shape)
    CS = plt.contour(X, Y, P)
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

In [ ]:

def Estep(mu, sigma, phi):
    # calculate determinants of sigma's
    det_sigma = np.array([[np.linalg.det(sigma[i])] 
                          for i in range(k)])
    # calculate inverse matrices for sigma's
    inv_sigma = np.array([np.linalg.inv(sigma[i]) 
                          for i in range(k)]).reshape(sigma.shape)
    # calculate Q(z) = p(x|z)*p(z)/p(x)
    pxz = np.array([
            np.exp(
                -0.5*np.diagonal(
                    np.dot(
                        np.dot(X_train - mu[i,:], inv_sigma[i]), 
                        np.transpose(X_train - mu[i,:])
                    )
                )
            )/((2.0*np.pi)**(n/2.0)*det_sigma[i,0]**0.5)*phi[i,0] 
            for i in range(k)]).T
    pz = pxz/np.sum(pxz, axis=1).reshape((-1, 1))
    return pz

def Mstep(pz):
    pz_sum = np.sum(pz, axis=0).reshape((-1,1))
    # update parameters
    phi_new = pz_sum/m
    mu_new = np.transpose(np.dot(X_train.T, pz)/pz_sum.T)
    sigma_new = np.array([
            np.dot(np.array([
                        np.outer(X_train[j,:] - mu_new[i,:], 
                                 X_train[j,:] - mu_new[i,:]) 
                        for j in range(m)]).reshape((m, -1)).T, 
                   pz[:,i]).reshape((n,n))/pz_sum[i,0] 
            for i in range(k)]).reshape((k,n,n))
    return mu_new, sigma_new, phi_new

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1)
ax.plot(X_train[:,0], X_train[:,1], 'o')
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

In [ ]:

# number of components
k = 2
# number of features
n = X_train.shape[1]
# number of training examples
m = X_train.shape[0]
# number of iterations
niter = 10
# initial values of phi
phi = np.array([1.0/k]*k).reshape((k,-1))
# initial values for mu and sigma
mu = []
sigma = []
np.random.seed(234)
for cl in range(k):
    mu.append(np.mean(X_train[np.random.choice(int(m), int(m/2)),:], axis=0))
    sigma.append(np.identity(n))
mu = np.array(mu).reshape((k, n))
sigma = np.array(sigma).reshape((k, n, n))

In [ ]:
for nit in range(niter):
    print('Iteration ' + str(nit+1) + ':')
    pz = Estep(mu, sigma, phi)
    mu, sigma, phi = Mstep(pz)
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(X_train[:,0], X_train[:,1], 'o', alpha=.1, ms=5)
    for i in range(k):    
        mu_i = mu[i,:]
        sigma_i = sigma[i]
        sigma_i_inv = np.linalg.inv(sigma_i)
        x = np.linspace(-15.0, 15.0)
        y = np.linspace(-4.0, 10.0)
        X, Y = np.meshgrid(x, y)
        XX = np.array([X.ravel(), Y.ravel()]).T
        P = np.exp(-0.5*np.diagonal(
                np.dot(
                    np.dot(XX - mu_i, sigma_i_inv), 
                    np.transpose(XX - mu_i)
                )))/(2*np.pi*np.linalg.det(sigma_i)**0.5)
        P = P.reshape(X.shape)
        CS = plt.contour(X, Y, P)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.show()

## 3D Visualizing

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.datasets import make_gaussian_quantiles
from sklearn.datasets import make_classification, make_regression
from sklearn.externals import six
import pandas as pd
import numpy as np
import argparse
import json
import re
import os
import sys
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode()

def rename_columns(df, prefix='x'):
    """
    Rename the columns of a dataframe to have X in front of them

    :param df: data frame we're operating on
    :param prefix: the prefix string
    """
    df = df.copy()
    df.columns = [prefix + str(i) for i in df.columns]
    return df

In [ ]:
X, Y = make_classification(n_samples=100, n_classes=3, n_features=3, n_redundant=0, n_informative=3,
                             scale=1000, n_clusters_per_class=1)
df = pd.DataFrame(X)
# rename X columns
df = rename_columns(df)
# and add the Y
df['y'] = Y
df.head(3)

In [ ]:

cluster1=df.loc[df['y'] == 0]
cluster2=df.loc[df['y'] == 1]
cluster3=df.loc[df['y'] == 2]

scatter1 = dict(
    mode = "markers",
    name = "Cluster 1",
    type = "scatter3d",    
    x = cluster1.as_matrix()[:,0], y = cluster1.as_matrix()[:,1], z = cluster1.as_matrix()[:,2],
    marker = dict( size=2, color='green')
)
scatter2 = dict(
    mode = "markers",
    name = "Cluster 2",
    type = "scatter3d",    
    x = cluster2.as_matrix()[:,0], y = cluster2.as_matrix()[:,1], z = cluster2.as_matrix()[:,2],
    marker = dict( size=2, color='blue')
)
scatter3 = dict(
    mode = "markers",
    name = "Cluster 3",
    type = "scatter3d",    
    x = cluster3.as_matrix()[:,0], y = cluster3.as_matrix()[:,1], z = cluster3.as_matrix()[:,2],
    marker = dict( size=2, color='red')
)
cluster1 = dict(
    alphahull = 5,
    name = "Cluster 1",
    opacity = .1,
    type = "mesh3d",    
    x = cluster1.as_matrix()[:,0], y = cluster1.as_matrix()[:,1], z = cluster1.as_matrix()[:,2],
    color='green', showscale = True
)
cluster2 = dict(
    alphahull = 5,
    name = "Cluster 2",
    opacity = .1,
    type = "mesh3d",    
    x = cluster2.as_matrix()[:,0], y = cluster2.as_matrix()[:,1], z = cluster2.as_matrix()[:,2],
    color='blue', showscale = True
)
cluster3 = dict(
    alphahull = 5,
    name = "Cluster 3",
    opacity = .1,
    type = "mesh3d",    
    x = cluster3.as_matrix()[:,0], y = cluster3.as_matrix()[:,1], z = cluster3.as_matrix()[:,2],
    color='red', showscale = True
)
layout = dict(
    title = 'Interactive Cluster Shapes in 3D',
    scene = dict(
        xaxis = dict( zeroline=True ),
        yaxis = dict( zeroline=True ),
        zaxis = dict( zeroline=True ),
    )
)
fig = dict( data=[scatter1, scatter2, scatter3, cluster1, cluster2, cluster3], layout=layout )
# Use py.iplot() for IPython notebook
plotly.offline.iplot(fig, filename='mesh3d_sample')